In [13]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse

#RegEx
import regex as re

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:


#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, \
case when (contains(r.observations, 'CR') )  then 1 else 0 end as CR \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US','en_GB','en_CA') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and r.created_date > '2020-12-21' \
and mg.usage_type != 'Test' \
and ugc_id in (322273839, 326270105) \
"

#and ugc_id in (315642957,276609384,273955886, 285059791)
#and contains(r.observations, 'CR') \
#and mg.name in ('MidwayUSA', 'Midsouth Shooters','Ulta') and ugc_id in (315642957,276609384,273955886, 285059791) \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UGC_ID           2 non-null      int64 
 1   NAME             2 non-null      object
 2   REVIEW_HEADLINE  2 non-null      object
 3   REVIEW_COMMENTS  2 non-null      object
 4   CR               2 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 208.0+ bytes

In [20]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ... CR
0  326270105  ...  0
1  322273839  ...  0

[2 rows x 5 columns]

In [21]:
reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS']

reviews['NAME_1ST_PART'] = reviews['NAME'].apply(lambda x: x.split()[0])

#Lower case
reviews['ALL_TEXT'] = reviews['ALL_TEXT'].str.lower()
reviews['NAME_1ST_PART'] = reviews['NAME_1ST_PART'].str.lower()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#dfTest['bar'] = dfTest.apply(lambda row : row['foo'].replace(str(row['date']), ''), axis=1)
reviews['ALL_TEXT_2'] = reviews.apply(lambda x: str(x['ALL_TEXT']).replace(str(x['NAME_1ST_PART']),''), axis = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#Ignore where midwayusa client name bc target references don't refer to Target the retailer but target as in target practice
#Can't look for case sensitivity bc people still start a sentence with target and therefore it's capitalized
#Midsouth Shooters as well, Cheaper Than Dirt as well
#3Rivers Archery, JoeBobOutfitters - Terminated, Leupold - TERMINATED 
#reviews['ALL_TEXT_3'] = reviews.apply(lambda x: str(x['ALL_TEXT_2']).replace('target','') if (x['NAME']=='MidwayUSA' or x['NAME']=='Midsouth Shooters' or x['NAME']=='Cheaper Than Dirt' or x['NAME']=='3Rivers Archery' or x['NAME']=='JoeBobOutfitters - Terminated' or x['NAME']=='Leupold - TERMINATED' ) else x['ALL_TEXT_2'], axis = 1)

#Ignore where Citi card holders mention Costco bc it's a Costco related credit card
#reviews['ALL_TEXT_4'] = reviews.apply(lambda x: str(x['ALL_TEXT_3']).replace('costco','') if (x['NAME']=='Citigroup Inc.') else x['ALL_TEXT_3'], axis = 1)

#Ignore where sears in mentioned in BBQ Guys reviews, almost always refering to searing of meat
#reviews['ALL_TEXT_5'] = reviews.apply(lambda x: str(x['ALL_TEXT_4']).replace('sears','') if (x['NAME']=='BBQ Guys') else x['ALL_TEXT_4'], axis = 1)

#Marshalls is a brand sold at Wickes, also at Guitar Center, and at Musician's Friend, Sam Ash
#Books-A-Million has series with Marshalls in the name
#reviews['ALL_TEXT_6'] = reviews.apply(lambda x: str(x['ALL_TEXT_5']).replace('marshalls','') if (x['NAME']=='Wickes Building Supplies' or x['NAME']=='Guitar Center' or x['NAME']=="Musician's Friend" or x['NAME']=='Sam Ash' or x['NAME']=='Books-A-Million') else x['ALL_TEXT_5'], axis = 1)
#reviews['ALL_TEXT_7'] = reviews.apply(lambda x: str(x['ALL_TEXT_6']).replace("marshall's",'') if (x['NAME']=='Wickes Building Supplies' or x['NAME']=='Guitar Center' or x['NAME']=="Musician's Friend" or x['NAME']=='Sam Ash' or x['NAME']=='Books-A-Million') else x['ALL_TEXT_6'], axis = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:

#Waitrose is owned by John Lewis - so shouldn't be marked for John Lewis - add to regex list or no?
#Remove john lewis, big lots
#Remove Goodwill - doesn't count as retailer
#If on or on- before target then ignore, lots of references to products being on target for size or whatever and not refering to retailer Target
#(?:(?<!ON[ -]))TARGET
test['match'] = reviews['ALL_TEXT_2'].str.extractall(r'\b(AMAZON(?! (?:PAY|ALEXA|FIRE STICK))|E[ -]?BAY|OVERSTOCK|WAL[ -]?MART|(?:(?<!ON[ -]))TARGET|SEARS|ACE|LOWE\'?S|HOME DEPOT|MENARD\'?S|B ?& ?Q|WICKES|CABELA\'?S|FLEET FARM|DOLLAR TREE|DOLLAR GENERAL|FAMILY DOLLAR|SAM\'?S|COSTCO|MACY\'?S|NORDSTROM|JC[ ]?PENNE?Y|TJ[ -]?MAXX?|MARSHALL\'?S|(?:(?<!BOB ))ROSS|KOHL\'?S|J[ ]?CREW|CVS|WALGREEN\'?S|RITE[ -]?AID|QVC|HSN|KROGER|WHOLE FOODS|JEWEL OSCO|TJ\'?S|TRADER JOE\'?S?|WEGMAN\'?S|ALBERTSON|ALDI\'?S?|TESCO(?:(?! (?:CLUB[ ]?CARD|POINTS|VOUCHERS)))|CARREFOUR|SEPHORA|POTTERY BARN|IKEA|BED BATH & BEYOND|BED BATH AND BEYOND|WAYFAIR|ZAPPOS|FOOT LOCKER|DSW|AUTO[ ]?ZONE|ETSY|HOBBY LOBBY|MICHAEL\'?S|BLICK|DICK\'?S|REI|AETNA|STATE FARM|IPSY)\b', flags=re.I)

#Use this to create all one row, otherwise above returns multi rows for each mention found per ugc_id
#.unstack().apply(lambda x:','.join(x.dropna()), axis=1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
test

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              0   match
  match                
0 0       blick   blick
  1       blick   blick
1 0      amazon  amazon
  1      amazon  amazon

In [25]:
reviews['model_CR'] = 0

#Look for where regex_retailer is not null and return 1
reviews.loc[(reviews['regex_retailer'].isna() ), 'model_CR'] = 1

#Then look for specific clients and return 0

#Ignore where midwayusa client name bc target references don't refer to Target the retailer but target as in target practice
#Can't look for case sensitivity bc people still start a sentence with target and therefore it's capitalized
#Midsouth Shooters as well, Cheaper Than Dirt as well
#3Rivers Archery, JoeBobOutfitters - Terminated, Leupold - TERMINATED 
reviews.loc[((reviews['NAME']=='MidwayUSA') |(reviews['NAME']=='Midsouth Shooters') | (reviews['NAME']=='Cheaper Than Dirt')| (reviews['NAME']=='3Rivers Archery')| (reviews['NAME']=='JoeBobOutfitters - Terminated')| (reviews['NAME']=='Leupold - TERMINATED'))&(reviews['regex_retailer']=='target'), 'model_CR'] = 0
#Ignore where Citi card holders mention Costco bc it's a Costco related credit card
reviews.loc[(reviews['NAME']=='Citigroup Inc.') &(reviews['regex_retailer']=='costco'), 'model_CR'] = 0
#Ignore where sears in mentioned in BBQ Guys reviews, almost always refering to searing of meat
reviews.loc[(reviews['NAME']=='BBQ Guys') &(reviews['regex_retailer']=='sears'), 'model_CR'] = 0
#Marshalls is a brand sold at Wickes, also at Guitar Center, and at Musician's Friend, Sam Ash
#Books-A-Million has series with Marshalls in the name
reviews.loc[((reviews['NAME']=='Wickes Building Supplies')|(reviews['NAME']=='Guitar Center')|(reviews['NAME']=="Musician's Friend")|(reviews['NAME']=='Sam Ash')|(reviews['NAME']=='Books-A-Million')) &((reviews['regex_retailer']=='marshalls')|(reviews['regex_retailer']=="marshall's")), 'model_CR'] = 0
#Dick Blick
reviews.loc[(reviews['NAME']=='Dick Blick') &(reviews['regex_retailer']=='blick'), 'model_CR'] = 0
#Consumer Reports allows retailer mentions so no CR label
reviews.loc[(reviews['NAME']=='Consumer Reports'), 'model_CR'] = 0


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'regex_retailer'
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/frame.py", line 3024, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/indexes/base.py", line 3082, in get_loc
    raise KeyError(key) from err
KeyError: 'regex_retailer'



In [26]:
reviews

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ... model_CR
0  326270105  ...        0
1  322273839  ...        0

[2 rows x 9 columns]

In [ ]:
reviews['retailer_not_client'] = pd.notnull(reviews['regex_retailer'])

In [ ]:
#Join case insensitive and sensitive columns into one
#reviews['regex_retailer'] = reviews[['regex_retailer_insens_case', 'regex_retailer_sens_case']].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)

#Replace empty strings from above with nan
#reviews['regex_retailer'] = reviews['regex_retailer'].replace('', np.nan)

#Check for nan which means no retailer mention and returns True where there is a retailer mention
#reviews['retailer_not_client'] = pd.notnull(reviews['regex_retailer'])

In [ ]:
reviews.iloc[:,-4:].head(100)

In [ ]:
reviews['model_CR'] = 0

reviews.loc[(reviews['retailer_not_client']==True ), 'model_CR'] = 1

reviews.loc[(reviews['NAME']=='Dick Blick') & (reviews['regex_retailer']=='blick') , 'model_CR'] = 0
#Consumer Reports allows retailer mentions so no CR label
reviews.loc[(reviews['NAME']=='Consumer Reports'), 'model_CR'] = 0


In [ ]:
FP = reviews[(reviews['CR'] == 0) & (reviews['model_CR'] == 1)]
FN = reviews[(reviews['CR'] == 1) & (reviews['model_CR'] == 0)]
TP = reviews[(reviews['CR'] == 1) & (reviews['model_CR'] == 1)]
TN = reviews[(reviews['CR'] == 0) & (reviews['model_CR'] == 0)]


In [ ]:

FP['UGC_ID'].count()

In [ ]:

FN['UGC_ID'].count()

In [ ]:

TP['UGC_ID'].count()

In [ ]:
TN['UGC_ID'].count()

In [ ]:
#FN.head(100)

In [ ]:
FN[['NAME','UGC_ID']].groupby(['NAME']).count().reset_index().sort_values(['UGC_ID'], ascending=False).head(10)

In [27]:
check=FP[FP['NAME']=='Wickes Building Supplies']
check.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          UGC_ID                      NAME  \
6176   298784319  Wickes Building Supplies   
6180   298769417  Wickes Building Supplies   
10275  268936291  Wickes Building Supplies   
13188  276869568  Wickes Building Supplies   
37761  317659440  Wickes Building Supplies   
37765  317500678  Wickes Building Supplies   
44190  277427048  Wickes Building Supplies   
44351  277396076  Wickes Building Supplies   
44381  277425575  Wickes Building Supplies   
44472  277414019  Wickes Building Supplies   

                                                    REVIEW_HEADLINE  \
6176   OK Castors                                                     
6180   Great product great price and no delay for delivery can coll   
10275  Wickes need to learn the correct way to store wood             
13188  Incomplete delivery and no follow-up from Wickes               
37761  Well done Wickes own brand                                     
37765  Paint                                                       

In [41]:
FP.groupby(['regex_retailer']).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                                UGC_ID  NAME  \
regex_retailer                                                                 
ace                                                             222     222    
ace,ace                                                         35      35     
ace,ace,ace                                                     2       2      
ace,ace,ace,ace                                                 1       1      
aetna                                                           16      16     
aetna,aetna                                                     1       1      
aetna,aetna,aetna                                               3       3      
aetna,aetna,aetna,aetna                                         1       1      
albertson                                                       10      10     
aldi                                                            67      67     
aldi's                                  

In [23]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("CR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("NAME_1ST_PART", StringType(), True)\
,StructField("regex_retailer_insens_case", StringType(), True)\
,StructField("regex_retailer_sens_case", StringType(), True)\
,StructField("regex_retailer", StringType(), True)\
,StructField("retailer_not_client", StringType(), True)\
,StructField("model_CR", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FN = spark.createDataFrame(FN, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
FN.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CR_FN_test3").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…